In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

# Import the API key.
from config import weather_api_key
from config import gkey

# Load in csv
WeatherPy_database_df = pd.read_csv("weather_data/WeatherPy_database.csv")
WeatherPy_database_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Pocone,BR,-16.26,-56.62,68.25,89,0,3.04,clear sky,0.00,0.0
1,1,Shingu,JP,33.73,135.98,80.60,78,75,5.82,broken clouds,0.00,0.0
2,2,Richards Bay,ZA,-28.78,32.04,63.07,73,4,12.95,clear sky,0.00,0.0
3,3,Punta Arenas,CL,-53.15,-70.92,30.20,86,0,11.41,clear sky,0.00,0.0
4,4,East London,ZA,-33.02,27.91,58.51,67,0,4.85,clear sky,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
548,548,Warmbad,NaN,-28.45,18.73,40.26,78,0,1.25,clear sky,0.00,0.0
549,549,Coahuayana,MX,18.73,-103.68,76.05,86,100,1.32,overcast clouds,0.00,0.0
550,550,Imeni Babushkina,RU,59.76,43.13,60.28,93,100,7.72,moderate rain,1.28,0.0
551,551,Aranos,NaN,-24.13,19.12,33.78,89,0,1.86,clear sky,0.00,0.0


In [2]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Prompt the customer to answer if he or she would like it to be raining or not.
rain_q = input("Do you want it to be raining? (yes/no) ") == "yes"
# Prompt the customer to answer if he or she would like it to be snowing or not.
snow_q = input("Do you want it to be snowing? (yes/no) ") == "yes"


What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [3]:
# Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not
if rain_q == True:
    rain_filter = WeatherPy_database_df["Rain (inches)"] > 0
else:
    rain_filter = WeatherPy_database_df["Rain (inches)"] == 0

if snow_q == True:
    snow_filter = WeatherPy_database_df["Snow (inches)"] > 0
else:
    snow_filter = WeatherPy_database_df["Snow (inches)"] == 0


In [4]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = WeatherPy_database_df.loc[
    (WeatherPy_database_df["Max Temp"]<= max_temp)
    & (WeatherPy_database_df["Max Temp"]>= min_temp)
    & rain_filter
    & snow_filter
]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Pocone,BR,-16.26,-56.62,68.25,89,0,3.04,clear sky,0.0,0.0
6,6,Quelimane,MZ,-17.88,36.89,66.20,93,0,2.84,clear sky,0.0,0.0
10,10,Bambous Virieux,MU,-20.34,57.76,71.60,94,75,12.75,shower rain,0.0,0.0
16,16,Quatre Cocos,MU,-20.21,57.76,71.60,94,75,12.75,shower rain,0.0,0.0
31,31,La Ronge,CA,55.10,-105.28,69.80,49,90,4.70,overcast clouds,0.0,0.0
33,33,Albany,US,42.60,-73.97,71.01,88,0,3.91,clear sky,0.0,0.0
37,37,Avarua,CK,-21.21,-159.78,71.60,78,4,8.05,light rain,0.0,0.0
40,40,Belyy Yar,RU,53.60,91.39,68.00,77,100,2.24,overcast clouds,0.0,0.0
42,42,Faya,SA,18.39,42.45,73.40,28,41,5.82,scattered clouds,0.0,0.0
47,47,Bethel,US,41.37,-73.41,69.01,68,1,4.70,clear sky,0.0,0.0


In [5]:
len(preferred_cities_df)

107

In [6]:
preferred_cities_df.count()

City_ID                107
City                   107
Country                107
Lat                    107
Lng                    107
Max Temp               107
Humidity               107
Cloudiness             107
Wind Speed             107
Current Description    107
Rain (inches)          107
Snow (inches)          107
dtype: int64

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pocone,BR,68.25,clear sky,-16.26,-56.62,
6,Quelimane,MZ,66.20,clear sky,-17.88,36.89,
10,Bambous Virieux,MU,71.60,shower rain,-20.34,57.76,
16,Quatre Cocos,MU,71.60,shower rain,-20.21,57.76,
31,La Ronge,CA,69.80,overcast clouds,55.10,-105.28,
33,Albany,US,71.01,clear sky,42.60,-73.97,
37,Avarua,CK,71.60,light rain,-21.21,-159.78,
40,Belyy Yar,RU,68.00,overcast clouds,53.60,91.39,
42,Faya,SA,73.40,scattered clouds,18.39,42.45,
47,Bethel,US,69.01,clear sky,41.37,-73.41,


In [8]:
# Set the parameters to search for a hotel.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey,
    "location": ["Lat", "Lng"]}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pocone,BR,68.25,clear sky,-16.26,-56.62,Hotel e Churrascaria Pantaneira
6,Quelimane,MZ,66.20,clear sky,-17.88,36.89,Hotel Flamingo
10,Bambous Virieux,MU,71.60,shower rain,-20.34,57.76,Casa Tia Villa
16,Quatre Cocos,MU,71.60,shower rain,-20.21,57.76,Veranda Palmar Beach Hotel
31,La Ronge,CA,69.80,overcast clouds,55.10,-105.28,Northland Motor Hotel
33,Albany,US,71.01,clear sky,42.60,-73.97,
37,Avarua,CK,71.60,light rain,-21.21,-159.78,Paradise Inn
40,Belyy Yar,RU,68.00,overcast clouds,53.60,91.39,Pasportnyy Stol
42,Faya,SA,73.40,scattered clouds,18.39,42.45,مقبرة ال مجمل
47,Bethel,US,69.01,clear sky,41.37,-73.41,Courtyard by Marriott Danbury


In [11]:
# Add a pop-up marker for each city that displays the hotel name, city name, country, and maximum temperature.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [12]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the hotel_df into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")